In [16]:
%%html
<link rel="stylesheet" type="text/css" href="./files/style.css">

# UCL BIOC0016 - Bioinformatics
Alan R. Lowe (a.lowe@ucl.ac.uk)

---


### Bioimage informatics / Machine learning

The aim of the exercises in this notebook is to familiarise you with the different steps of evaluating the performance of a simple convolutional neural network (CNN). You will use a real CNN to classify different cell states using image data provided to you. A more sophisticated version of this neural network has been used in recently published research. The architecture of the network is similar to those you will have learnt about in the lectures.

The network is able to tell whether a cell is proliferating or dead.  This could be useful to understand whether a drug is effective or not, or to understand the normal cellular behaviour.

The CNN has been pre-trained on a dataset of real images from data collected at UCL. This is a very simple CNN, and you are going to assess the performance of it by making predictions of the cell state using image data, and comparing these with your ground truth annotation.  The images that you are provided with are called a 'hold out' set, since they have come from the much larger original data set, but have not been used to train the neural network. The CNN has not 'seen' these data during the training phase, and therefore represent a real test of the performance of the network.  

The practical contains following sections:

1. Data annotation
2. Make predictions with a convolutional neural network
3. Compare these with the data that you have annotated
4. Determine the accuracy of the model

### Instructions

Code sections are coloured according to the following scheme:

* <div class="task_red"> Code that needs to be written by you. </div>
* <div class="task_blue"> Code that needs to be edited by you, perhaps changing some parameters. </div>
* <div class="task_green"> A task that needs to be completed by you. This may be recording the results. </div>


### Be part of the research project!

This is based on a real research project, and you can contribute to the project by recording your results as part of the practical. You can read more about the research project [here](http://lowe.cs.ucl.ac.uk/cellx.html).

As you complete different sections of the practical, we ask you to complete different sections of a google form. By recording the results, you can contribute to the research programme.

### Further reading
* http://lowe.cs.ucl.ac.uk/cellx.html
* https://en.wikipedia.org/wiki/Bioimage_informatics
* https://teachablemachine.withgoogle.com/train/image
* https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226

---
# PRACTICAL


---
## PART 1 - Data Annotation


To test how well the neural network is able to perform on unseen images, we need to manually annotate some new and unseen images with a label. We can then count how many labels the neural network correctly predicts. In this section, you will be provided with a random sample of unlabeled images. You will need at least 30, although the more the better. Using the guide below, please annotate each image with one of the six labels provided. If you are unsure, use the 'Unknown' label.

Here are some examples of cells and their corresponding labels:

![Cell state labels](./files/cell_states.png "Cell state labels")

### Set up plotting and import some useful libraries

We first need to load some libraries of code that will help with the data processing and visualization. If you are more experienced at Python, you can read the code for the `helpers` library at [github](https://github.com/quantumjot/BIOC0016-MachineLearning).

In [ ]:
%matplotlib inline

import helpers
import numpy as np

### Get a sample of images from the test dataset

Next, we need to get a random sample of images from the dataset. The dataset has thousands of images in it. The first command `images = helpers.get_example_images()` uses the `helpers` library to call a function `get_example_images`. This requests images from the dataset and places them in a list called `images`.

The second command `helpers.plot_images()` takes the list of images as an argument and plots them in a matrix, showing the unique ID numbers above them.

In [ ]:
images = helpers.get_example_images(num_images=10)
helpers.plot_images(images)

<div class="task_blue"> <b>TASK</b>: Try changing the number of images. For the remainder of the practical, you will need at least 30 images.</div>

Now that you have your own sample of images, you need to annotate them to create the **ground truth**. You will store these annotations using two Python data structures, a `list` and a `dictionary`.

A `list` is just what it sounds like, a sequence of items in order. A list can have multiple entries (separated by commas), or be empty:
```python
list_of_numbers = [0, 1, 2, 3, 4]
empty_list = []
```

A `dictionary` stores key-value pairs.

The final data structure may look something like this:

```python 
annotation = {'interphase': [0,5,3,20], 
              'prometaphase': [7,2], 
              'metaphase': [14,33,6], 
              'anaphase': [508,1], 
              'apoptosis': [8], 
              'unknown': []}```

<div class="task_red">
    <b>TASK:</b> Complete the annotation dictionary. To do this you will need to make a list of the image numbers, and your best guess as to the label that each image should have. Once you have this, compile the information into a dictionary structure, and run the validation to make sure that the annotation format is correct. If the validation returns True, you can copy the annotation cell to the <a href="https://docs.google.com/forms/d/e/1FAIpQLSdhPIHCMvtHVUCe-7_XYHujWIW2mq7SzcA2mfYh6LHiZ2LUPQ/viewform">Google form</a>.          
</div>

In [ ]:
annotation = {'prometaphase': [893]}

In [ ]:
# validate the annotation
helpers.validate_annotation(annotation)

<div class="task_green">
    <b>TASK:</b> Complete the <a href="https://docs.google.com/forms/d/e/1FAIpQLSdhPIHCMvtHVUCe-7_XYHujWIW2mq7SzcA2mfYh6LHiZ2LUPQ/viewform">Google form</a>, to record the annotations you have made
</div>

<br />

If your annotation has validated `True`, you can now assign these annotations to the images:

In [ ]:
helpers.annotate_images(images, annotation)

---
## PART 2 - Using the CNN to make predictions

### Loading the pre-trained model

In the following lines of code `helpers.load_CNN_model()` builds the CNN network, and sets the weights and biases using the pretrained values.

In [ ]:
model = helpers.load_CNN_model()

<div class='task_red'> <b>TASK:</b> Summarise the model and get the number of parameters. Answer the following questions:
    <ul>
        <li>How many convolutional layers are there? </li>
        <li>How many kernels are used in each convolutional layer? </li>
        <li>How many output classes are there? </li>
        <li>What is the size of the input image, and after the convolutional layers?</li>
    </ul>
</div>

**HINT**: You can use the command `model.summary()` to get the details of the model.

In [ ]:
model.summary()

### Visualise activations within the network

When the network is given an example, we can visualize the activations within the networks from the sample. To visualize the activations within the network, you can use the `helpers.visualize_layers()` command.

To do this, the function needs three (3) arguments:
`helpers.visualize_layers(model, image, layer)`

<br />

<div class='task_red'><b>TASK:</b> visualize the activations within the network for the first and second convolutional layers (layer=0 or layer=1). What can you see? </div>

In [ ]:
helpers.visualize_layers(model, images[5], layer=0)

### Make the predictions


Now that we have loaded the pre-trained model. We can feed it the same images that you have labeled and generate predictions for the label. The predictions are returned as the probability of the label given the image data, $\text{P}(\text{label} | \text{data})$. In fact, the model returns the probability for *all* labels given the data.

<br />
<div class="task_blue"> <b>TASK:</b> Using the model, make predictions for the images that you have annotated.</div>

In [ ]:
images_for_prediction = np.concatenate([im.as_tensor() for im in images], axis=0)
predictions = model.predict(images_for_prediction)

You can use the command `print()` to print the predictions to the screen.  Since `predictions` is a 2D array, it also has a shape. You can access the shape information using the `.shape` property.

<br />

<div class='task_red'><b>TASK:</b> Print the predictions to the screen. Why is the output this shape?</div>

---
## PART 3 - Compare the predictions with your annotations

We can visualize the images and their predictions using one of the helper functions:

In [ ]:
helpers.plot_predictions(images, predictions)

In [ ]:
helpers.print_predictions(images, predictions)

### Calculate the confusion matrix

By definition, a confusion matrix $\mathbf{C}$ is such that $\mathbf{C}_{i,j}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.

---
## PART 4 - Calculate the accuracy of the model using your ground truth data

We need to determine the overall accuracy of the model. We will make several simple calculations:

That is, precision is the fraction of events where we correctly declared i out of all instances where the algorithm declared i. Conversely, recall is the fraction of events where we correctly declared i out of all of the cases where the true of state of the world is i.


#### Accuracy

Accuracy is the proportion of true results among the total number of examples examined.

#### Precision

Precision is 

#### Recall

#### F1-score

We will now write our own function to calculate the F1-score for each of the categories.

A *function* is a block of code that takes some inputs (*arguments*) and performs a task, often returning a result. For example, a function that calculates the value of $y$ for a straight line $y=mx+c$ may look something like this:

```python
def straight_line(m, x, c):
    y = m*x + c
    return y
```

The `def` keyword means that we are defining a function. This is followed by the function name, which is `straight_line` in this case, with the arguments `(m, x, c)` in that order. The remainder is simply the calculation. The `return` keyword, returns the result `y` to the user.

One can use the function as follows:
```python
y = straight_line(1, 10, 0)
print(y)
```

which returns the result: `10`

You will need to create a function below, that calculates the $F_{1}$ score, according to the following equation:

\begin{equation}
F_1 = 2 \times \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}
\end{equation}

<br />

<div class='task_red'><b>TASK:</b> Complete the function below to calculate the F1 score for each class </div>

In [ ]:
def F1_score(recall, precision):
    """ Function to calculate the F1-score. You need to complete the function here. """

    return score


<div class="task_red"><b>TASK:</b> Please make sure you have completed the Google form and uploaded your results </div>

# End of the practical
---